## 0. Setting

In [ ]:
import pandas as pd
import geopandas as gpd
import os, gc
from tqdm import tqdm
import re, glob
import numpy as np

os.chdir(os.getcwd())

def print_save(file, t, a='a'):
    file = '1_log/' + file
    with open(file, a) as f:
        f.write(f"\n{t}")
    print(t)

## 2.108 preprocess

In [ ]:
# Firm transaction data (108)

INPUT_PATH  = "data/raw/VATTXDT0108/VATTXDT0108.txt"  # if .txt, use sep='|'
SEP         = '|' if INPUT_PATH.lower().endswith('.txt') else ','
CHUNK_SIZE  = 10000000
LOG_PATH    = "1_log/1_108_preprocess.txt"

# Years to process: list for specific years, None for all
YEARS       = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
TOTAL_ROWS  = 2073201182

ALL_COLS = ['1_BZNO','2_BASE_YY','3_PRD_CCD','4_SALE_BUY_CCD','5_VAT_ITM_KCD','6_VAT_SEQ',
            '7_CONO_PID','8_KEDCD','9_CO_BZNO_CCD','10_TX_PRD_FROM','11_TX_PRD_TO','12_TX_PRD_SUTB_YN',
            '13_TXPL_TXRI_RNK','14_TXRI','15_TXPL_BZNO','16_TXPL_PID','17_TXPL_KEDCD','18_TXPL_NM',
            '19_TXPL_TCN','20_TXPL_TSPPR','21_TXPL_TTAX_AM','22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID','24_BZNO_del','None']
USECOLS = ['2_BASE_YY','4_SALE_BUY_CCD','22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID']
DTYPES  = {'2_BASE_YY':'int32','4_SALE_BUY_CCD':'int8','22_BZNO_SIMSID':'object','23_TXPL_BZNO_SIMSID':'object'} 

OUT_DIR = "data/processed/deal_network"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)

# Buffers for tracking yearly results
curr_year = None
year_rows_total_buf = 0            # rows after year filter, before dropna
year_rows_kept_buf  = 0            # rows kept after dropna
year_pairs_buf      = set()        # unique (src, dst) pairs for the current year
per_year_rows_total = {}           # total rows per year
per_year_rows_kept  = {}           # kept rows per year
years_touched       = set()        # years processed

rows_seen = 0
n_chunks  = 0

# Reader setup
reader = pd.read_csv(
    INPUT_PATH, sep=SEP, header=0, names=ALL_COLS, usecols=USECOLS,
    dtype=DTYPES, chunksize=CHUNK_SIZE, low_memory=False
)
total_iter = (TOTAL_ROWS // CHUNK_SIZE + 1) if TOTAL_ROWS else None

# Helper to write yearly outputs
def flush_year(y, merge_pairs):
    out_path = os.path.join(OUT_DIR, f"filtered_108_{y}.csv")
    df_out = pd.DataFrame(merge_pairs, columns=['22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID'])
    if os.path.exists(out_path):
        prev = pd.read_csv(out_path, dtype = {'2_BASE_YY':'int32', '22_BZNO_SIMSID': str, '23_TXPL_BZNO_SIMSID': str})
        prev_pairs = set(prev[['22_BZNO_SIMSID', '23_TXPL_BZNO_SIMSID']].itertuples(index=False, name=None))
        union_pairs = prev_pairs | merge_pairs
        df_out = pd.DataFrame(union_pairs, columns=['22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID'])

    df_out.insert(0,'2_BASE_YY', y)
    df_out.to_csv(out_path, index=False)

# Main chunk loop
for chunk in tqdm(reader, total=total_iter, desc="Processing chunks"):
    n_chunks  += 1
    rows_seen += len(chunk)

    # Year filter
    if YEARS is not None:
        chunk = chunk[chunk['2_BASE_YY'].isin(YEARS)]
    if chunk.empty:
        gc.collect()
        continue

    # Iterate through years within chunk (boundary-safe)
    for y in chunk['2_BASE_YY'].astype('int32').drop_duplicates().to_numpy():
        y = int(y)

        # Flush if year changes
        if curr_year is None:
            curr_year = y
        elif y != curr_year:
            if year_rows_total_buf > 0:
                flush_year(curr_year, year_pairs_buf)
                year_rows_total_buf = 0
                year_rows_kept_buf  = 0
                year_pairs_buf.clear()
                gc.collect()
            curr_year = y

        # Subset for current year
        sub0 = chunk.loc[chunk['2_BASE_YY'] == y,
                         ['22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID','4_SALE_BUY_CCD']]
        year_rows_total_buf += len(sub0)
        years_touched.add(y)
        per_year_rows_total[y] = per_year_rows_total.get(y, 0) + len(sub0)
    
        sub = sub0.dropna()
        kept = len(sub)
        year_rows_kept_buf += kept
        per_year_rows_kept[y] = per_year_rows_kept.get(y, 0) + kept

        # Normalize: swap SIMSID for purchases (2)
        mask_in = (sub['4_SALE_BUY_CCD'] == 2)
        if mask_in.any():
            tmp = sub.loc[mask_in, '22_BZNO_SIMSID'].to_numpy(copy=True)
            sub.loc[mask_in, '22_BZNO_SIMSID'] = sub.loc[mask_in, '23_TXPL_BZNO_SIMSID'].to_numpy(copy=True)
            sub.loc[mask_in, '23_TXPL_BZNO_SIMSID'] = tmp

        # Add unique pairs
        year_pairs_buf.update(sub[['22_BZNO_SIMSID','23_TXPL_BZNO_SIMSID']].itertuples(index=False, name=None))

        del sub0, sub

    del chunk
    gc.collect()

# Flush last year
if curr_year is not None and year_rows_total_buf > 0:
    flush_year(curr_year, year_pairs_buf)
    year_pairs_buf.clear()
    gc.collect()

# Summary log
final_years = sorted(years_touched)
per_year_unique_pairs = {}
for y in final_years:
    p = os.path.join(OUT_DIR, f"filtered_108_{y}.csv")
    if os.path.exists(p):
        try:
            n = max(0, sum(1 for _ in open(p, 'r', encoding = 'utf-8')) - 1)
        except Exception:
            df_tmp = pd.read_csv(p, dtype = {'2_BASE_YY':'int32', '22_BZNO_SIMSID': str, '23_TXPL_BZNO_SIMSID': str})
            n = len(df_tmp)
            del df_tmp
        per_year_unique_pairs[y] = n
    else:
        per_year_unique_pairs[y] = 0

with open(LOG_PATH, 'w', encoding = 'utf-8') as f:
    f.write("Preprocess summary (dataset 108)\nPer year:\n")
    all_rows = 0
    all_filtered_rows = 0
    all_pairs = 0
    for y in final_years:
        total_ = per_year_rows_total.get(y,0)
        all_rows += total_
        kept_ = per_year_rows_kept.get(y,0)
        all_filtered_rows += kept
        uniq_ = per_year_unique_pairs.get(y,0)
        all_pairs += uniq_
        f.write(f"  - Year {y}: rows = {total_:,}, filtered_rows = {kept_:,}, unique_pairs = {uniq_:,}\n")
    
    f.write(f"\nAll rows = {rows_seen:,}, 2016-2022 rows = {all_rows:,}, all_filtered_rows = {all_filtered_rows}, all_pairs = {uniq_}")


## 3. Sims preprocess

In [ ]:
D001 = pd.read_sas("data/raw/sims_d001.sas7bdat", format='sas7bdat', encoding='ansi')
D001.columns=['1_사업자등록번호','2_법인등록번호','3_우편번호','4_설립일자','5_휴업폐업일자','6_표준산업분류항목코드','7_표준산업분류 대분류','8_주요생산품내용','9_법인여부','10_기업공개코드','11_기업상태코드','12_기업형태코드','13_기업규모코드','14_우수기업여부','15_사업자성격구분코드']

#1m

In [ ]:
df = D001
print_save('1_SIMS_preprocess.txt', 'SIMS_preprocess', a= 'w')


# read spatial info
gdfs = []
sd = os.listdir('data/raw/원본파일(시도별)')
for map in sd:
    gdf = gpd.read_file(f'data/raw/원본파일(시도별)/{map}/TL_KODIS_BAS.shp', encoding='cp949')
    gdfs.append(gdf)
BAS_map = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
BAS_map[['BAS_ID', 'SIG_CD', 'SIG_KOR_NM']] = BAS_map[['BAS_ID', 'SIG_CD', 'SIG_KOR_NM']].astype(str)
BAS_map = BAS_map[['BAS_ID', 'SIG_CD', 'SIG_KOR_NM']]

# merge spatial info
print_save('1_SIMS_preprocess.txt', f'\nBAS_ID missing rate: {round(df["3_우편번호"].isna().mean(), 3)}')
df = df.merge(BAS_map, left_on='3_우편번호', right_on='BAS_ID', how='left')

# filter error data
df['4_설립일자'] = pd.to_datetime(df['4_설립일자'], format='%Y%m%d', errors='coerce')
df['5_휴업폐업일자'] = pd.to_datetime(df['5_휴업폐업일자'], format='%Y%m%d', errors='coerce')

df = df[['1_사업자등록번호', '4_설립일자', '6_표준산업분류항목코드','7_표준산업분류 대분류','13_기업규모코드', 'SIG_CD', 'SIG_KOR_NM']]
df.to_csv("data/processed/deal_network/filtered_SIMS.csv", index=False)

#30s

## 4.Stats preprocess

In [ ]:
stat_D = pd.read_csv('data/raw/250714_중소기업 지원정책 수립 지원을 위한 자료제공_수정_DATA_ENC.txt', sep = "\t",  dtype={
                'BRNO': str,
                'ADMDST_CLSF_CD': str,
                "ENT_SCL_SE_CD": str,
                "BZENT_KSIC_LVL5_CD": str
            })

#1m

In [ ]:
stat_D_filter = stat_D[['CRTR_1', 'BRNO', 'ENT_SCL_SE_CD', 'BZENT_KSIC_LVL5_CD']]
stat_D_filter.columns = ['기준년도', '1_사업자등록번호', '13_기업규모코드_stat', '6_표준산업분류항목코드_stat']
stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='2', '13_기업규모코드_stat'] = '1'

stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='4', '13_기업규모코드_stat'] = '2'
stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='5', '13_기업규모코드_stat'] = '2'

stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='6', '13_기업규모코드_stat'] = '4'

stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='91', '13_기업규모코드_stat']= '9999'
stat_D_filter = stat_D_filter.sort_values('기준년도', ascending=False)
stat_D_filter = stat_D_filter.groupby('1_사업자등록번호')[['1_사업자등록번호','13_기업규모코드_stat', '6_표준산업분류항목코드_stat']].bfill()
stat_D_filter = stat_D_filter.drop_duplicates('1_사업자등록번호')
stat_D_filter['6_표준산업분류항목코드_stat'] = stat_D_filter['6_표준산업분류항목코드_stat'].fillna('9999').astype(int)


conditions = [
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 1) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <=3),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 5) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <=8),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 10) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 34),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) == 35),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 36) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 39),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 41) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 42),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 45) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 47),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 49) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 52),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 55) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 56),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 58) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 63),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 64) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 66),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) == 68),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 70) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 73),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 74) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 76),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) == 84),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) == 85),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 86) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 87),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 90) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 91),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 94) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 96),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) >= 97) & (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) <= 98),
  (stat_D_filter['6_표준산업분류항목코드_stat'].astype(str).str[:2].astype(int) == 99)
]

values = ['A' ,'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']

stat_D_filter['7_표준산업분류 대분류_stat'] = np.select(conditions, values, default = None)

stat_D_filter.loc[stat_D_filter['6_표준산업분류항목코드_stat']=='9999', '6_표준산업분류항목코드_stat']= None
stat_D_filter.loc[stat_D_filter['13_기업규모코드_stat']=='9999', '13_기업규모코드_stat']= None
stat_D_filter = stat_D_filter[['1_사업자등록번호', '13_기업규모코드_stat', '6_표준산업분류항목코드_stat', '7_표준산업분류 대분류_stat']]
stat_D_filter['6_표준산업분류항목코드_stat'] = stat_D_filter['6_표준산업분류항목코드_stat'].astype(int).astype(str)
stat_D_filter.to_csv("data/processed/deal_network/filtered_Stat.csv", index=False)

#3m

## 5. geo_network

### 5.1. merge

In [ ]:
# read files

deal_files = sorted(glob.glob("data/processed/deal_network/filtered_108_*.csv"))

stat_D_filter = pd.read_csv(
    "data/processed/deal_network/filtered_Stat.csv",
    dtype = {
        '1_사업자등록번호': 'string',
        "13_기업규모코드_stat": str,
        "6_표준산업분류항목코드_stat": str
    },
)


SIMS = pd.read_csv(
    "data/processed/deal_network/filtered_SIMS.csv",
    dtype={
        '1_사업자등록번호': 'string',
        '6_표준산업분류항목코드': str,
        '13_기업규모코드': str,
        'SIG_CD': str,
        'SIG_KOR_NM': str
    }
)

SIMS = SIMS[SIMS['SIG_CD'].notnull()]
#20s

In [ ]:
os.makedirs("data/processed/deal_network/deal_by/year_by", exist_ok=True)

print_save('1_geo_network.txt', f"\n\nstart", a='w')

missings_df = pd.DataFrame()

for year_file in deal_files:
    # 1) Load yearly file and merge seller/buyer information
    df = pd.read_csv(year_file, dtype={'2_BASE_YY': str, '22_BZNO_SIMSID': 'string', '23_TXPL_BZNO_SIMSID': str})
    year = int(df['2_BASE_YY'].mode().iloc[0]) if len(df) else -1
    
    IDset = set(list(df['22_BZNO_SIMSID'].unique()) + list(df['23_TXPL_BZNO_SIMSID'].unique()))
    stat_D_filter_temp = stat_D_filter[stat_D_filter['1_사업자등록번호'].isin(IDset)]
    SIMS_temp = SIMS[SIMS['1_사업자등록번호'].isin(IDset)]
    del IDset
    
    df = pd.merge(df, stat_D_filter_temp, how='left',
                  left_on='22_BZNO_SIMSID', right_on='1_사업자등록번호', suffixes=('_seller', '_buyer'))
    df = pd.merge(df, stat_D_filter_temp, how='left',
                  left_on='23_TXPL_BZNO_SIMSID', right_on='1_사업자등록번호', suffixes=('_seller', '_buyer'))
    df = df.drop(columns=['1_사업자등록번호_seller', '1_사업자등록번호_buyer'])
    
    df = pd.merge(df, SIMS_temp, how='left',
                  left_on='22_BZNO_SIMSID', right_on='1_사업자등록번호', suffixes=('_seller', '_buyer'))
    df = pd.merge(df, SIMS_temp, how='left',
                  left_on='23_TXPL_BZNO_SIMSID', right_on='1_사업자등록번호', suffixes=('_seller', '_buyer'))
    df = df.drop(columns=['1_사업자등록번호_seller', '1_사업자등록번호_buyer'])

    # Fill missing attributes with fallback (stat table → SIMS table)
    df['13_기업규모코드_seller'] = df['13_기업규모코드_stat_seller'].fillna(df['13_기업규모코드_seller'])
    df['6_표준산업분류항목코드_seller'] = df['6_표준산업분류항목코드_stat_seller'].fillna(df['6_표준산업분류항목코드_seller'])
    df['7_표준산업분류 대분류_seller'] = df['7_표준산업분류 대분류_stat_seller'].fillna(df['7_표준산업분류 대분류_seller'])
    df['13_기업규모코드_buyer'] = df['13_기업규모코드_stat_buyer'].fillna(df['13_기업규모코드_buyer'])
    df['6_표준산업분류항목코드_buyer'] = df['6_표준산업분류항목코드_stat_buyer'].fillna(df['6_표준산업분류항목코드_buyer'])
    df['7_표준산업분류 대분류_buyer'] = df['7_표준산업분류 대분류_stat_buyer'].fillna(df['7_표준산업분류 대분류_buyer'])
    
    # Missing-value statistics (stat table vs SIMS table)
    summ_missings = {
        'Columns' : ['기업규모코드_seller', '표준산업분류항목코드_seller', '표준산업분류 대분류_seller',
                     '기업규모코드_buyer', '표준산업분류항목코드_buyer', '표준산업분류 대분류_buyer'],
        'Stats'   : df[['13_기업규모코드_stat_seller', '6_표준산업분류항목코드_stat_seller',
                        '7_표준산업분류 대분류_stat_seller', '13_기업규모코드_stat_buyer',
                        '6_표준산업분류항목코드_stat_buyer', '7_표준산업분류 대분류_stat_buyer']].isnull().mean().values.tolist(),
        'Kodata' : df[['13_기업규모코드_seller', '6_표준산업분류항목코드_seller',
                       '7_표준산업분류 대분류_seller', '13_기업규모코드_buyer',
                       '6_표준산업분류항목코드_buyer', '7_표준산업분류 대분류_buyer']].isnull().mean().values.tolist()
    }
    
    summarry_stats = pd.DataFrame(summ_missings)
    summarry_stats['diff'] = summarry_stats['Kodata'] - summarry_stats['Stats']
    summarry_stats[['Stats', 'Kodata', 'diff']] = round(summarry_stats[['Stats', 'Kodata', 'diff']]*100, 1)
    summarry_stats['year'] = year
    missings_df = pd.concat([missings_df, summarry_stats])
    
    # Select and rename columns
    df = df[['2_BASE_YY', '22_BZNO_SIMSID', '23_TXPL_BZNO_SIMSID',
             '4_설립일자_seller','6_표준산업분류항목코드_seller','7_표준산업분류 대분류_seller','13_기업규모코드_seller',
             'SIG_CD_seller','SIG_KOR_NM_seller',
             '4_설립일자_buyer','6_표준산업분류항목코드_buyer','7_표준산업분류 대분류_buyer','13_기업규모코드_buyer',
             'SIG_CD_buyer','SIG_KOR_NM_buyer']]
    
    # 2) Derive year and firm age
    df['2_BASE_YY'] = pd.to_datetime(df['2_BASE_YY'], format='%Y', errors='coerce').dt.year
    df['4_설립일자_seller'] = pd.to_datetime(df['4_설립일자_seller'], errors='coerce').dt.year
    df['4_설립일자_buyer']  = pd.to_datetime(df['4_설립일자_buyer'], errors='coerce').dt.year
    df['기업연령_seller'] = df['2_BASE_YY'] - df['4_설립일자_seller']
    df['기업연령_buyer']  = df['2_BASE_YY'] - df['4_설립일자_buyer']

    # 3) Final selection & renaming
    df = df[['2_BASE_YY',
             '기업연령_seller','6_표준산업분류항목코드_seller','7_표준산업분류 대분류_seller','13_기업규모코드_seller',
             'SIG_CD_seller','SIG_KOR_NM_seller',
             '기업연령_buyer','6_표준산업분류항목코드_buyer','7_표준산업분류 대분류_buyer','13_기업규모코드_buyer',
             'SIG_CD_buyer','SIG_KOR_NM_buyer']]
    df.columns = ['1_기준연도',
                  '4_기업연령_seller','5_표준산업분류항목코드_seller','6_표준산업분류 대분류_seller','7_기업규모코드_seller',
                  '8_시군구코드_seller','9_시군구명_seller',
                  '10_기업연령_buyer','11_표준산업분류항목코드_buyer','12_표준산업분류 대분류_buyer','13_기업규모코드_buyer',
                  '14_시군구코드_buyer','15_시군구명_buyer']

    # 4) Data type adjustments (int vs str)
    int_cols = ['1_기준연도','4_기업연령_seller','10_기업연령_buyer']
    str_cols = ['5_표준산업분류항목코드_seller','11_표준산업분류항목코드_buyer',
                '7_기업규모코드_seller','13_기업규모코드_buyer',
                '8_시군구코드_seller','14_시군구코드_buyer']

    df[int_cols] = df[int_cols].fillna(9999).astype(int)
    df[str_cols] = df[str_cols].fillna('9999').astype(str)

    # 5) Map firm size labels and bin firm ages
    label_map = {'1':'대기업','2':'중소기업','3':'중견기업','4':'소상공인'}
    df['7_기업규모코드_seller'] = df['7_기업규모코드_seller'].map(label_map).fillna('9999')
    df['13_기업규모코드_buyer'] = df['13_기업규모코드_buyer'].map(label_map).fillna('9999')

    df['4_기업연령_seller'] = pd.cut(df['4_기업연령_seller'],
                                  bins=[-1,1,5,10,100],
                                  labels=['1년 미만','1~5년 미만','5~10년 미만','10년 이상'])
    df['10_기업연령_buyer']  = pd.cut(df['10_기업연령_buyer'],
                                  bins=[-1,1,5,10,100],
                                  labels=['1년 미만','1~5년 미만','5~10년 미만','10년 이상'])

    # 6) Group definitions
    innovation_codes = ['21','261','2621','263','264','265','266','284','285','27','282','313']
    urban_codes = ['C','F','G','H','I','J','K','L','M','N','P','Q','R','S']

    groups = []
    groups.append(('all', None))
    groups.append(('man', (df['6_표준산업분류 대분류_seller']=='C') & (df['12_표준산업분류 대분류_buyer']=='C')))
    groups.append(('innovation',
        (df['5_표준산업분류항목코드_seller'].str.startswith(tuple(innovation_codes))) &
        (df['11_표준산업분류항목코드_buyer'].str.startswith(tuple(innovation_codes)))
    ))
    for age_group in df['4_기업연령_seller'].dropna().unique():
        groups.append((f'urban_age_{age_group}',
            (df['6_표준산업분류 대분류_seller'].isin(urban_codes)) &
            (df['4_기업연령_seller']==age_group) &
            (df['10_기업연령_buyer']==age_group)))
    for size in set(df['7_기업규모코드_seller'].dropna().unique()) - {'9999'}:
        groups.append((f'urban_size_{size}',
            (df['6_표준산업분류 대분류_seller'].isin(urban_codes)) &
            (df['7_기업규모코드_seller']==size) &
            (df['13_기업규모코드_buyer']==size)))

    # 7) Derive year directly from data (not filename)
    year = int(df['1_기준연도'].mode().iloc[0]) if len(df) else -1
    print_save('1_geo_network.txt', f"\n\n====== {year} ======")

    # 8) Loop through groups, mask, and save per year
    for group_name, group_con in groups:
        print_save('1_geo_network.txt', f"\n====== {group_name} ======")

        df_group = df if group_con is None else df[group_con]

        # Count total transactions by (year, seller SGG, buyer SGG)
        df_temp = (
            df_group
            .groupby(['1_기준연도','8_시군구코드_seller','14_시군구코드_buyer'], dropna=False)
            .size()
            .reset_index(name='총거래관계')
        )
        df_temp['총거래관계'] = df_temp['총거래관계'].astype(int)

        print_save('1_geo_network.txt', f'original links: {len(df_temp)}')

        # Save per-year network files
        out_pref = f"data/processed/deal_network/deal_by/year_by/network_by_{group_name}_{year}"
        df_temp.to_csv(out_pref + ".csv", index=False)
        
missings_df[['year','Columns','Stats','Kodata','diff']].to_csv("data/processed/deal_network/size_ind_missings.csv", index=False)

# 30m

In [ ]:
year_dir = "data/processed/deal_network/deal_by/year_by"
out_dir  = "data/processed/deal_network/deal_by"
os.makedirs(out_dir, exist_ok=True)

# Use only per-year original files from year_by (exclude masked versions)
paths = [p for p in glob.glob(os.path.join(year_dir, "network_by_*_*.csv"))]

# Extract group and year from filenames (group = all text before the last underscore)
rows = []
for p in paths:
    fname = os.path.basename(p)
    m = re.match(r"network_by_(.+)_(\d{4})\.csv$", fname)
    if m:
        rows.append((p, m.group(1), int(m.group(2))))
idx = pd.DataFrame(rows, columns=["path","group","year"])

if idx.empty:
    print("No yearly network files found:", year_dir)

for g in sorted(idx["group"].unique()):
    sub = idx[idx["group"] == g]

    parts = []
    for p in sub["path"]:
        df_y = pd.read_csv(
            p,
            dtype={
                "1_기준연도": "int64",
                "8_시군구코드_seller": str,
                "14_시군구코드_buyer": str,
                "총거래관계": "int64",
            },
        )
        # Keep only required columns
        df_y = df_y[["1_기준연도","8_시군구코드_seller","14_시군구코드_buyer","총거래관계"]]
        parts.append(df_y)

    if not parts:
        continue

    df_all = pd.concat(parts, ignore_index=True)

    # Aggregate OD flows including year
    df_agg = (
        df_all
        .groupby(["1_기준연도","8_시군구코드_seller","14_시군구코드_buyer"], dropna=False)["총거래관계"]
        .sum()
        .reset_index()
        .astype({"1_기준연도":"int64","총거래관계":"int64"})
        .sort_values(["1_기준연도","8_시군구코드_seller","14_시군구코드_buyer"])
    )

    # Save aggregated results (with year included)
    out = os.path.join(out_dir, f"network_by_{g}.csv")
    df_agg.to_csv(out, index=False)
